In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    IntegerType,
    ArrayType,
    DateType,
    BooleanType,
    TimestampType,
    DoubleType,
)
from pyspark.sql.functions import (
    col,
    to_timestamp,
    to_date,
    when,
    broadcast,
    count,
    sum as spark_sum,
    count_distinct,
)
from pyspark.sql.window import Window

In [17]:
spark = (
    SparkSession.builder.appName("Revenue Analysis")
    .master("local[8]")
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")
    # .config("spark.hadoop.fs.defaultFS", "hdfs://127.0.0.1:9000")
    .getOrCreate()
)

spark_context = spark.sparkContext.getOrCreate()

print("\n===== Spark Context Info =====")
print(f"App Name      : {spark_context.appName}")
print(f"Master        : {spark_context.master}")
print(f"Application ID: {spark_context.applicationId}")
print(f"UI Web URL    : {spark_context.uiWebUrl}")
print(f"Version       : {spark_context.version}")
print(f"Python Ver    : {spark_context.pythonVer}")


===== Spark Context Info =====
App Name      : Revenue Analysis
Master        : local[8]
Application ID: local-1758562105877
UI Web URL    : http://172.20.10.14:4040
Version       : 4.0.1
Python Ver    : 3.11


In [18]:
!head -n 1 data/items.jsonl | jq "keys_unsorted" 
!head -n 1 data/users.jsonl | jq "keys_unsorted" 

[
  "item_id",
  "category",
  "tags"
]
[
  "id",
  "signup_date",
  "plan",
  "country",
  "marketing_opt_in"
]


In [19]:
!gunzip -c data/events/part-00.jsonl.gz | head -n 1 | jq "."

{
  "ts": "2025-08-23T13:43:20.968573+00:00",
  "event": "view",
  "user_id": 12482,
  "item_id": 3686,
  "context": {
    "country": "US",
    "device": "android",
    "locale": "pt_BR",
    "session_id": "bdd640fb-0667-4ad1-9c80-317fa3b1799d"
  },
  "props": {
    "price": null,
    "payment_method": null,
    "dwell_ms": 1232
  },
  "exp": {
    "ab_group": "B"
  }
}
gunzip: error writing to output: Broken pipe
gunzip: data/events/part-00.jsonl.gz: uncompress failed


In [20]:
items_schema = StructType(
    [
        StructField("item_id", IntegerType(), False),
        StructField("category", StringType(), False),
        StructField("tags", ArrayType(StringType()), False),
    ]
)

user_schema = StructType(
    [
        StructField("id", IntegerType(), False),
        StructField("signup_date", DateType(), False),
        StructField("plan", StringType(), False),
        StructField("country", StringType(), False),
        StructField("marketing_opt_in", BooleanType(), False),
    ]
)

event_schema = StructType(
    [
        StructField("ts", TimestampType(), False),
        StructField("event", StringType(), False),
        StructField("user_id", IntegerType(), False),
        StructField("item_id", IntegerType(), False),
        StructField(
            "context",
            StructType(
                [
                    StructField("country", StringType(), False),
                    StructField("device", StringType(), False),
                    StructField("locale", StringType(), False),
                    StructField("session_id", StringType(), False),
                ]
            ),
        ),
        StructField(
            "props",
            StructType(
                [
                    StructField("price", DoubleType(), True),
                    StructField("payment_method", StringType(), True),
                    StructField("dwell_ms", IntegerType(), True),
                ]
            ),
        ),
        StructField(
            "exp",
            StructType(
                [
                    StructField("ab_group", StringType(), True),
                ]
            ),
        ),
    ]
)

In [21]:
df_users = spark.read.schema(user_schema).json("data/users.jsonl")
df_items = spark.read.schema(items_schema).json("data/items.jsonl")
df_events = spark.read.schema(event_schema).json(
    [
        "data/events/part-00.jsonl.gz",
        "data/events/part-01.jsonl.gz",
        "data/events/part-02.jsonl.gz",
        "data/events/part-03.jsonl.gz",
    ]
)


print(
    f"users | total - {df_users.count()} | partitions - {df_users.rdd.getNumPartitions()}"
)
df_users.show(5)
print(
    f"\nitems | total - {df_items.count()} | partitions - {df_items.rdd.getNumPartitions()}"
)
df_items.show(5)

users | total - 20000 | partitions - 1
+---+-----------+----+-------+----------------+
| id|signup_date|plan|country|marketing_opt_in|
+---+-----------+----+-------+----------------+
|  1| 2024-09-29|free|     TH|           false|
|  2| 2025-04-30|free|     DE|            true|
|  3| 2024-08-08| pro|     DE|            true|
|  4| 2025-01-18|free|     DE|           false|
|  5| 2025-04-25|free|     US|            true|
+---+-----------+----+-------+----------------+
only showing top 5 rows

items | total - 5000 | partitions - 1
+-------+-----------+--------------------+
|item_id|   category|                tags|
+-------+-----------+--------------------+
|      1|     sports|    [new, clearance]|
|      2|     sports|[sale, popular, c...|
|      3|     sports|   [new, gift, sale]|
|      4|      books|              [gift]|
|      5|electronics|     [sale, popular]|
+-------+-----------+--------------------+
only showing top 5 rows


In [22]:
df_events = spark.read.schema(event_schema).json(
    [
        "data/events/part-00.jsonl.gz",
        "data/events/part-01.jsonl.gz",
        "data/events/part-02.jsonl.gz",
        "data/events/part-03.jsonl.gz",
    ]
)

print(
    f"events | total - {df_events.count()} | partitions - {df_events.rdd.getNumPartitions()}"
)
df_events.show(5)

events | total - 200000 | partitions - 4
+--------------------+-----+-------+-------+--------------------+------------------+---+
|                  ts|event|user_id|item_id|             context|             props|exp|
+--------------------+-----+-------+-------+--------------------+------------------+---+
|2025-08-23 20:43:...| view|  12482|   3686|{US, android, pt_...|{NULL, NULL, 1232}|{B}|
|2025-08-23 20:43:...| view|   2361|   3322|{GB, ios, vi_VN, ...|{NULL, NULL, 4895}|{A}|
|2025-08-23 20:43:...| view|  17249|   3108|{US, web, fr_FR, ...|{NULL, NULL, 3663}|{A}|
|2025-08-23 20:43:...| view|  16719|   2772|{US, web, th_TH, ...|{NULL, NULL, 4178}|{A}|
|2025-08-23 20:45:...| view|   1937|     69|{IN, web, fr_FR, ...|{NULL, NULL, 2148}|{A}|
+--------------------+-----+-------+-------+--------------------+------------------+---+
only showing top 5 rows


In [23]:
df_events = df_events.withColumn("timestamp", to_timestamp("ts")).withColumn(
    "date", to_date("ts")
)

In [24]:
df_events = (
    df_events.withColumn(
        "revenue",
        when(col("event") == "purchase", col("props.price").cast(DoubleType()))
        .otherwise(0.0)
        .cast(DoubleType()),
    )
).filter(col("revenue") >= 0.0)

print(
    f"events | total - {df_events.count()} | partitions - {df_events.rdd.getNumPartitions()}"
)

events | total - 199995 | partitions - 4


#### broadcast

In [25]:
# spark shuffles both side of joins
# shuffles -> computationally expension and i/o operations included
# broadcast -> ships small tables to every executors
# now executors can join locally with their respective partitions

df_joined = df_events.join(broadcast(df_items), on="item_id", how="left").join(
    broadcast(df_users), df_events.user_id == df_users.id, how="left"
)

df_joined.explain("extended")

== Parsed Logical Plan ==
Join LeftOuter, (user_id#527 = id#450)
:- Project [item_id#528, ts#525, event#526, user_id#527, context#529, props#530, exp#531, timestamp#580, date#581, revenue#582, category#456, tags#457]
:  +- Join LeftOuter, (item_id#528 = item_id#455)
:     :- Filter (revenue#582 >= 0.0)
:     :  +- Project [ts#525, event#526, user_id#527, item_id#528, context#529, props#530, exp#531, timestamp#580, date#581, cast(CASE WHEN (event#526 = purchase) THEN cast(props#530.price as double) ELSE 0.0 END as double) AS revenue#582]
:     :     +- Project [ts#525, event#526, user_id#527, item_id#528, context#529, props#530, exp#531, timestamp#580, to_date(ts#525, None, Some(Asia/Bangkok), true) AS date#581]
:     :        +- Project [ts#525, event#526, user_id#527, item_id#528, context#529, props#530, exp#531, to_timestamp(ts#525, None, TimestampType, Some(Asia/Bangkok), true) AS timestamp#580]
:     :           +- Relation [ts#525,event#526,user_id#527,item_id#528,context#529,prop

In [26]:
df_aggregate = df_joined.groupBy("date", "country", "category").agg(
    count("*").alias("total_events"),
    count(when(col("event") == "purchase", 1)).alias("num_purchases"),
    spark_sum("revenue").alias("total_revenue"),
    count_distinct("user_id").alias("unique_users"),
)

print(
    f"aggregations | total - {df_aggregate.count()} | partitions - {df_aggregate.rdd.getNumPartitions()}"
)
df_aggregate.show(5)

aggregations | total - 3368 | partitions - 1
+----------+-------+--------+------------+-------------+------------------+------------+
|      date|country|category|total_events|num_purchases|     total_revenue|unique_users|
+----------+-------+--------+------------+-------------+------------------+------------+
|2025-08-26|     DE|   books|         190|            7|1194.5700000000002|         180|
|2025-09-02|     TH|   books|         123|            1|             93.62|         120|
|2025-09-10|     DE|    toys|         105|            2|            261.39|         102|
|2025-09-04|     VN|    toys|          97|            6|            792.43|          97|
|2025-09-11|     FR|   books|          84|            2|            103.65|          82|
+----------+-------+--------+------------+-------------+------------------+------------+
only showing top 5 rows


In [27]:
w = Window.partitionBy("country", "category").orderBy("date").rowsBetween(-6, 0)

df_final = df_aggregate.withColumn("revenue_7d", spark_sum("total_revenue").over(w))

In [28]:
# one folder per date partitionBy('date')

df_final.write.mode("overwrite").partitionBy("date").parquet("out/daily_kpi/")

In [29]:
df_sample = spark.read.parquet("out/daily_kpi/date=2025-10-23")

print(
    f"out/daily_kpi/date=2025-10-23 | total - {df_sample.count()} | partitions - {df_sample.rdd.getNumPartitions()}"
)
df_sample.show(10, truncate=False)

out/daily_kpi/date=2025-10-23 | total - 48 | partitions - 1
+-------+-----------+------------+-------------+-------------+------------+-----------------+
|country|category   |total_events|num_purchases|total_revenue|unique_users|revenue_7d       |
+-------+-----------+------------+-------------+-------------+------------+-----------------+
|BR     |books      |11          |0            |0.0          |11          |66.28            |
|BR     |electronics|9           |0            |0.0          |9           |197.13           |
|BR     |fashion    |11          |0            |0.0          |10          |261.17           |
|BR     |home       |6           |0            |0.0          |6           |989.4699999999999|
|BR     |sports     |9           |0            |0.0          |9           |473.81           |
|BR     |toys       |8           |0            |0.0          |8           |319.13           |
|DE     |books      |13          |0            |0.0          |13          |57.47            |


In [30]:
# TODO: repartition findings